In [218]:
from text_vectorian import SpBertVectorian
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import  numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Activation
from keras.models import Model
from keras import Input
import datetime
import keras
from pyknp import Juman

### BERTによるベクトル化のためのインポート
https://qiita.com/hideki/items/1ec1c21c33326ad5615f
のものをお借りした。

In [129]:
vectorizer_filename = f'./bert-wiki-ja/model.ckpt-1400000'
tokenizer_filename = f'./bert-wiki-ja/wiki-ja.model'
vectorian = SpBertVectorian(
    tokenizer_filename=tokenizer_filename,
    vectorizer_filename=vectorizer_filename
)

### 単語難易度辞書を作成。
データは
https://pj.ninjal.ac.jp/corpus_center/bccwj/freq-list.html
のものをお借りした。
pathは適当に合わせてください

In [187]:
vocab = pd.read_csv("./Datas/教科書.txt",encoding="utf8",sep="\t")
vocab = vocab[["語彙素","語彙素読み","初出学年"]]
grade_dif_dic = {"小_前":0,"小_後":1,"中":2,"高":3}
vocab_diff_dic = {}
for line in vocab.values:
    if line[0] in vocab_diff_dic and grade_dif_dic[line[2]] < vocab_diff_dic[line[0]]:
        vocab_diff_dic[line[0]] = grade_dif_dic[line[2]]
    else:
        vocab_diff_dic[line[0]] = grade_dif_dic[line[2]]

In [212]:
indep = ["名詞","動詞","形容詞","副詞"]

### modelのロード
my_model4.h5が学習済みモデルなので、ひとまずそれを使えば動きます。

In [217]:
model = keras.models.load_model("./my_model4.h5", compile=False)

### Score計算用関数の準備

In [260]:
def texts_to_inputs(text1,text2):
    t_l = text1
    t_r = text2

    t_l = vectorian.fit(t_l).vectors
    t_r = vectorian.fit(t_r).vectors

    itp = range(t_l.shape[1])
    in_x = np.insert(t_r,itp,t_l[:,itp],axis=1).reshape(1,256,768,1)
    return in_x

def easy_trans_scores(test1,test2,vocab_diff_dic):
    test_x = texts_to_inputs(test1,test2)
    p = model.predict(test_x)

    result = jumanpp.analysis(test2)
    d1 = 0
    d1_cnt = 0
    for mrph in result.mrph_list(): # 各形態素にアクセス
        if mrph.hinsi in indep:
            if mrph.genkei in vocab_diff_dic:
                d1 += vocab_diff_dic[mrph.genkei]
                d1_cnt += 1
    print("含意確率:{:.3g} 難易度合計:{} 自立語数:{} 平均難易度:{}".format(p[0][1],d1,d1_cnt,d1/d1_cnt))
    print("total score:{:.5g}".format(p[0][1]/d1))

### 実行例

In [281]:
test1 = "次の会議のために議題を書類に要約しておいてください．"
test2 = "次の会議のために話し合うことを紙にまとめておいてください．"
easy_trans_scores(test1,test2,vocab_diff_dic)

含意確率:0.994 難易度合計:0 自立語数:4 平均難易度:0.0
total score:inf


/Users/kakeru/PycharmProjects/main_env/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


In [282]:
test1 = "次の会議のために議題を書類に要約しておいてください"
test2 = "次の会議のためにアジェンダをレジュメにサマリーしといてください"
easy_trans_scores(test1,test2,vocab_diff_dic)

含意確率:0.778 難易度合計:9 自立語数:4 平均難易度:2.25
total score:0.086415


### 今のスコアリングだとうまく行かない例
難易度が0になると含意関係なくinfになる。
なんか良いスコアを持ってこなくては。

In [289]:
test1 = "これはお試し用の文章であって、本実験に利用する文章とは異なります"
test2 = "これは美味しいりんごです."
easy_trans_scores(test1,test2,vocab_diff_dic)

含意確率:0.0514 難易度合計:0 自立語数:1 平均難易度:0.0
total score:inf


/Users/kakeru/PycharmProjects/main_env/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
